In [10]:
from pydrake.examples.acrobot import (AcrobotGeometry, AcrobotInput,
                                      AcrobotPlant, AcrobotState, AcrobotParams)
from pydrake.all import Linearize, LinearQuadraticRegulator, SymbolicVectorSystem, Variable, Saturation, \
WrapToSystem, Simulator, Polynomial

from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

from IPython.display import SVG, display
import pydot
import numpy as np
import math
import control

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

import pydrake.forwarddiff as pf
import time

from IPython.display import display, Math, Markdown
from pydrake.examples.pendulum import PendulumPlant
from pydrake.examples.acrobot import AcrobotPlant
from pydrake.all import MultibodyPlant, Parser, SinCos, MakeVectorVariable, ToLatex, Substitute, Box
from underactuated import FindResource
from underactuated.scenarios import AddShape

from pydrake.all import (Jacobian, MathematicalProgram, Polynomial,
                         RegionOfAttraction, RegionOfAttractionOptions, Solve,
                         SymbolicVectorSystem, ToLatex, Variable, Variables,
                         plot_sublevelset_expression, AddMultibodyPlantSceneGraph, SolverOptions, 
                         CommonSolverOption)

from pydrake.solvers.mosek import MosekSolver
import sympy as sp
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

from IPython.display import display, Math
from pydrake.examples.pendulum import PendulumPlant
from pydrake.examples.acrobot import AcrobotPlant
from pydrake.all import MultibodyPlant, Parser, SinCos, MakeVectorVariable, ToLatex, Substitute, Box
from underactuated import FindResource
from underactuated.scenarios import AddShape

from pydrake.examples.quadrotor import (QuadrotorGeometry, QuadrotorPlant,
                                        StabilizingLQRController)

from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer


In [12]:
def PrintImplicitPolynomialDynamics(plant,
                                    context,
                                    sincos_indices):
    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()
    sym_context.SetTimeStateAndParametersFrom(context)
    sym_plant.FixInputPortsFrom(plant, context, sym_context)

    state = sym_context.get_continuous_state()
    derivatives = sym_context.Clone().get_mutable_continuous_state()

    q = MakeVectorVariable(state.num_q(), "q")
    v = MakeVectorVariable(state.num_v(), "v")
    qd = MakeVectorVariable(state.num_q(), "\dot{q}")
    vd = MakeVectorVariable(state.num_v(), "\dot{v}")

    state.SetFromVector(np.hstack((q, v)))
    derivatives.SetFromVector(np.hstack((qd, vd)))
    residual = sym_plant.CalcImplicitTimeDerivativesResidual(
        sym_context, derivatives)
        
    s = MakeVectorVariable(state.num_q(), "s")
    c = MakeVectorVariable(state.num_q(), "c")
    subs = {}
    for i in sincos_indices:
        subs[q[i]] = SinCos(s[i], c[i])
    
    dynamics = Substitute(residual, subs)
    dynamics = residual

    display(Markdown("$"+ToLatex(dynamics, 2) + "= 0$"))
    
    return dynamics, q, v, qd, vd, s, c


plant = Quadrotor2D()
context = plant.CreateDefaultContext()
plant.get_input_port().FixValue(context, [0, 0])
indices = [0, 1]
PrintImplicitPolynomialDynamics(plant, context, indices)

$\begin{bmatrix} ( - v_{0} + \dot{q}_{0}) \\ ( - v_{1} + \dot{q}_{1}) \\ ( - v_{2} + \dot{q}_{2}) \\ \dot{v}_{0} \\ (9.81 + \dot{v}_{1}) \\ \dot{v}_{2} \end{bmatrix}= 0$

(array([<Expression "( - v(0) + \dot{q}(0))">,
        <Expression "( - v(1) + \dot{q}(1))">,
        <Expression "( - v(2) + \dot{q}(2))">, <Expression "\dot{v}(0)">,
        <Expression "(9.8100000000000005 + \dot{v}(1))">,
        <Expression "\dot{v}(2)">], dtype=object),
 array([Variable('q(0)', Continuous), Variable('q(1)', Continuous),
        Variable('q(2)', Continuous)], dtype=object),
 array([Variable('v(0)', Continuous), Variable('v(1)', Continuous),
        Variable('v(2)', Continuous)], dtype=object),
 array([Variable('\dot{q}(0)', Continuous),
        Variable('\dot{q}(1)', Continuous),
        Variable('\dot{q}(2)', Continuous)], dtype=object),
 array([Variable('\dot{v}(0)', Continuous),
        Variable('\dot{v}(1)', Continuous),
        Variable('\dot{v}(2)', Continuous)], dtype=object),
 array([Variable('s(0)', Continuous), Variable('s(1)', Continuous),
        Variable('s(2)', Continuous)], dtype=object),
 array([Variable('c(0)', Continuous), Variable('c(1)', Contin

In [20]:
plant = QuadrotorPlant()
context = plant.CreateDefaultContext()
plant.get_input_port().FixValue(context, [0, 0, 0, 0])
context.get_mutable_continuous_state_vector()\
       .SetFromVector(np.array([0.0, 0.0, 2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

In [21]:
lin = Linearize(plant, context)

RuntimeError: The nominal operating point (x0,u0) is not an equilibrium point of the system.  Without additional information, a time-invariant linearization of this system is not well defined.